## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-17-22-58-10 +0000,nyt,"Republicans, Braced for Losses, Push More Voti...",https://www.nytimes.com/2026/02/17/us/politics...
1,2026-02-17-22-57-16 +0000,nyt,White House Shrugs Off Lutnick’s Epstein Ties,https://www.nytimes.com/2026/02/17/us/politics...
2,2026-02-17-22-54-00 +0000,wsj,Opinion | Europe Fills U.S. Aid Loss to Ukraine,https://www.wsj.com/opinion/ukraine-aid-europe...
3,2026-02-17-22-51-11 +0000,nyt,A Holiday High-School Game Is Shattered by a F...,https://www.nytimes.com/2026/02/17/us/rhode-is...
4,2026-02-17-22-50-00 +0000,wsj,Kenvue to Cut 3.5% of Workforce Ahead of Kimbe...,https://www.wsj.com/business/kenvue-to-cut-3-5...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,trump,30
51,jackson,23
50,jesse,19
148,new,17
175,talks,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
275,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,102
60,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,89
146,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,81
162,2026-02-17-17-49-41 +0000,nypost,Stephen Colbert rips CBS for pulling interview...,https://nypost.com/2026/02/17/media/colbert-ri...,78
229,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
275,102,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
60,78,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
229,77,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
75,58,2026-02-17-21-10-42 +0000,nypost,Inside the grueling training regimen for China...,https://nypost.com/2026/02/17/lifestyle/nyc-ch...
162,48,2026-02-17-17-49-41 +0000,nypost,Stephen Colbert rips CBS for pulling interview...,https://nypost.com/2026/02/17/media/colbert-ri...
196,42,2026-02-17-16-02-41 +0000,nypost,"Iran, US agree on ‘guiding principles’ of nucl...",https://nypost.com/2026/02/17/world-news/iran-...
20,36,2026-02-17-22-33-00 +0000,wsj,Mamdani Warns of Nearly 10% Property-Tax Boost...,https://www.wsj.com/us-news/mamdani-warns-of-n...
86,35,2026-02-17-20-52-07 +0000,bbc,UK police force assessing Stansted Airport pri...,https://www.bbc.com/news/articles/c98q4mz831mo...
28,32,2026-02-17-22-22-00 +0000,wsj,"Shooter Killed Ex-Wife, Son at Rhode Island Ho...",https://www.wsj.com/us-news/two-killed-three-i...
296,31,2026-02-17-04-34-07 +0000,nyt,"LA Mayor Karen Bass Says Casey Wasserman, 2028...",https://www.nytimes.com/2026/02/16/us/casey-wa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
